<a href="https://colab.research.google.com/github/jcplerm/courserapub/blob/master/ChicagoCrimes01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from bs4 import BeautifulSoup as bsoup
import requests
import pandas as pd
import numpy as np
import string

In [2]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


In [195]:
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Prepare data

Run this section if needed to re-create test datasets

Extract this file from google drive:

 /content/drive/MyDrive/Colab Notebooks/ExploratoryDataAnalysis/Crimes_-_2001_to_Present.csv.zip


First must mount google drive to the runtime.

In [21]:
from zipfile import ZipFile

with ZipFile('/content/drive/MyDrive/Colab Notebooks/ExploratoryDataAnalysis/Crimes_-_2001_to_Present.csv.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

Read the CSV file

In [22]:
crimes = pd.read_csv('/content/Crimes_-_2001_to_Present.csv')

In [29]:
crimes.head(10)

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,Date_str,Datetime,Day of Week,Hour,Month,Week of Year,Day of Year
0,10224738,HY411648,2015-09-05,043XX S WOOD ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,924,9.0,12.0,61.0,08B,1165074.0,1875917.0,2015,02/10/2018 03:50:01 PM,41.815117,-87.670000,"(41.815117282, -87.669999562)",09/05/2015 01:30:00 PM,2015-09-05 13:30:00,5,13,9,36,248
1,10224739,HY411615,2015-09-04,008XX N CENTRAL AVE,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,1511,15.0,29.0,25.0,06,1138875.0,1904869.0,2015,02/10/2018 03:50:01 PM,41.895080,-87.765400,"(41.895080471, -87.765400451)",09/04/2015 11:30:00 AM,2015-09-04 11:30:00,4,11,9,36,247
2,11646166,JC213529,2018-09-01,082XX S INGLESIDE AVE,0810,THEFT,OVER $500,RESIDENCE,False,True,631,6.0,8.0,44.0,06,NaN,NaN,2018,04/06/2019 04:04:43 PM,NaN,NaN,NaN,09/01/2018 12:01:00 AM,2018-09-01 00:01:00,5,0,9,35,244
3,10224740,HY411595,2015-09-05,035XX W BARRY AVE,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,1412,14.0,35.0,21.0,18,1152037.0,1920384.0,2015,02/10/2018 03:50:01 PM,41.937406,-87.716650,"(41.937405765, -87.716649687)",09/05/2015 12:45:00 PM,2015-09-05 12:45:00,5,12,9,36,248
4,10224741,HY411610,2015-09-05,0000X N LARAMIE AVE,0560,ASSAULT,SIMPLE,APARTMENT,False,True,1522,15.0,28.0,25.0,08A,1141706.0,1900086.0,2015,02/10/2018 03:50:01 PM,41.881903,-87.755121,"(41.881903443, -87.755121152)",09/05/2015 01:00:00 PM,2015-09-05 13:00:00,5,13,9,36,248
5,10224742,HY411435,2015-09-05,082XX S LOOMIS BLVD,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,False,False,614,6.0,21.0,71.0,05,1168430.0,1850165.0,2015,02/10/2018 03:50:01 PM,41.744379,-87.658431,"(41.744378879, -87.658430635)",09/05/2015 10:55:00 AM,2015-09-05 10:55:00,5,10,9,36,248
6,10224743,HY411629,2015-09-04,021XX W CHURCHILL ST,0620,BURGLARY,UNLAWFUL ENTRY,RESIDENCE-GARAGE,False,False,1434,14.0,32.0,24.0,05,1161628.0,1912157.0,2015,02/10/2018 03:50:01 PM,41.914636,-87.681631,"(41.914635603, -87.681630909)",09/04/2015 06:00:00 PM,2015-09-04 18:00:00,4,18,9,36,247
7,10224744,HY411605,2015-09-05,025XX W CERMAK RD,0860,THEFT,RETAIL THEFT,GROCERY FOOD STORE,True,False,1034,10.0,25.0,31.0,06,1159734.0,1889313.0,2015,09/17/2015 11:37:18 AM,41.851989,-87.689219,"(41.851988885, -87.689219118)",09/05/2015 01:00:00 PM,2015-09-05 13:00:00,5,13,9,36,248
8,10224745,HY411654,2015-09-05,031XX W WASHINGTON BLVD,0320,ROBBERY,STRONGARM - NO WEAPON,STREET,False,True,1222,12.0,27.0,27.0,03,1155536.0,1900515.0,2015,02/10/2018 03:50:01 PM,41.882814,-87.704326,"(41.88281374, -87.704325717)",09/05/2015 11:30:00 AM,2015-09-05 11:30:00,5,11,9,36,248
9,11645836,JC212333,2016-05-01,055XX S ROCKWELL ST,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,NaN,False,False,824,8.0,15.0,63.0,11,NaN,NaN,2016,04/06/2019 04:04:43 PM,NaN,NaN,NaN,05/01/2016 12:25:00 AM,2016-05-01 00:25:00,6,0,5,17,122


In [31]:
crimes.shape

(7338320, 29)

In [ ]:
crimes.dtypes

In [ ]:
crimes.isnull().sum()

ID                           0
Case Number                  4
Date                         0
Block                        0
IUCR                         0
Primary Type                 0
Description                  0
Location Description      8299
Arrest                       0
Domestic                     0
Beat                         0
District                    47
Ward                    614837
Community Area          613484
FBI Code                     0
X Coordinate             72896
Y Coordinate             72896
Year                         0
Updated On                   0
Latitude                 72896
Longitude                72896
Location                 72896
dtype: int64

## Create and save dataframe with the last 3 previous years

In [25]:
crimes['Date_str']=crimes['Date']
crimes['Datetime']=pd.to_datetime(crimes['Date_str'])
crimes['Day of Week']=crimes['Datetime'].dt.dayofweek
crimes['Date']=crimes['Datetime'].dt.date
crimes['Hour']=crimes['Datetime'].dt.hour

In [26]:
crimes['Month']=crimes['Datetime'].dt.month
crimes['Week of Year']=crimes['Datetime'].dt.weekofyear
crimes['Day of Year']=crimes['Datetime'].dt.dayofyear

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  


In [30]:
crimes.dtypes

ID                               int64
Case Number                     object
Date                            object
Block                           object
IUCR                            object
Primary Type                    object
Description                     object
Location Description            object
Arrest                            bool
Domestic                          bool
Beat                             int64
District                       float64
Ward                           float64
Community Area                 float64
FBI Code                        object
X Coordinate                   float64
Y Coordinate                   float64
Year                             int64
Updated On                      object
Latitude                       float64
Longitude                      float64
Location                        object
Date_str                        object
Datetime                datetime64[ns]
Day of Week                      int64
Hour                     

In [32]:
crimes.to_csv('/content/drive/MyDrive/Colab Notebooks/ExploratoryDataAnalysis/crimes_dates.csv')

In [33]:
crimes3yr = crimes.loc[(crimes['Year']>=2018) & (crimes['Year']<=2020)]

In [34]:
crimes3yr.to_csv('/content/drive/MyDrive/Colab Notebooks/ExploratoryDataAnalysis/crimes3yr.csv')

# Exploratory Analysis

# Possible explorations:

- Decompose datetime into separate fields
- Aggregate by year, location, primary type, description, etc
- Aggregate by day of week
- Build histogram of #crimes per day


- Crimes by day of week
- Stacked graph of crime types over years
- Stacked graph of crimes within a year
- Pie chart of  type totals
- Surface chart of crimes over coordinates
- Histogram of crime counts vs number of days

Future:
- Correlate crimes with social, economic and education data



In [230]:
crimes = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ExploratoryDataAnalysis/crimes3yr.csv')

# Plot crimes map

In [231]:
def plot_crimes(location,crimes,color='blue',map=None):
  if map is None:
    geolocator = Nominatim(user_agent="jlerm_explorer")
    location = geolocator.geocode(location)
    latitude = location.latitude
    longitude = location.longitude
    map = folium.Map(location=[latitude, longitude], zoom_start=10)

  if len(crimes)==0:
    return map
    
  for lat, lng, id in zip(crimes['Latitude'], crimes['Longitude'], crimes['ID']):
    label = id
    label = folium.Popup(label, parse_html=True)
    if (pd.notnull(lat)):
      folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color=color,
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map)  

  return map

In [234]:
#plot_crimes('Chicago,IL',crimes10k[crimes10k['Primary Type']=='NARCOTICS'])
m=plot_crimes('Chicago,IL',crimes[crimes['Description']=='LOOTING'].head(10000))

In [235]:
m

In [236]:
crimes.head(5)

,Unnamed: 0,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,Date_str,Datetime,Day of Week,Hour,Month,Week of Year,Day of Year
0,2,11646166,JC213529,2018-09-01,082XX S INGLESIDE AVE,0810,THEFT,OVER $500,RESIDENCE,False,True,631,6.0,8.0,44.0,06,NaN,NaN,2018,04/06/2019 04:04:43 PM,NaN,NaN,NaN,09/01/2018 12:01:00 AM,2018-09-01 00:01:00,5,0,9,35,244
1,90,12014684,JD189901,2020-03-17,039XX N LECLAIRE AVE,0820,THEFT,$500 AND UNDER,STREET,False,False,1634,16.0,45.0,15.0,06,1141659.0,1925649.0,2020,03/25/2020 03:45:43 PM,41.952052,-87.754660,"(41.952051946, -87.754660372)",03/17/2020 09:30:00 PM,2020-03-17 21:30:00,1,21,3,12,77
2,158,11645648,JC212959,2018-01-01,024XX N MONITOR AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,False,2515,25.0,30.0,19.0,11,NaN,NaN,2018,04/06/2019 04:04:43 PM,NaN,NaN,NaN,01/01/2018 08:00:00 AM,2018-01-01 08:00:00,0,8,1,1,1
3,183,11864018,JC476123,2019-09-24,022XX S MICHIGAN AVE,1154,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT $300 AND UNDER,COMMERCIAL / BUSINESS OFFICE,False,False,132,1.0,3.0,33.0,11,1177560.0,1889548.0,2019,10/20/2019 03:56:02 PM,41.852248,-87.623786,"(41.852248185, -87.623786256)",09/24/2019 08:00:00 AM,2019-09-24 08:00:00,1,8,9,39,267
4,235,11859805,JC471592,2019-10-13,024XX W CHICAGO AVE,0860,THEFT,RETAIL THEFT,GROCERY FOOD STORE,False,False,1221,12.0,26.0,24.0,06,1160005.0,1905256.0,2019,10/20/2019 04:03:03 PM,41.895732,-87.687784,"(41.895732399, -87.687784384)",10/13/2019 08:30:00 PM,2019-10-13 20:30:00,6,20,10,41,286


In [238]:
crimes_by_date=crimes.groupby(['Date']).count()

In [239]:
crimes_by_date.describe()

,Unnamed: 0,ID,Case Number,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,Date_str,Datetime,Day of Week,Hour,Month,Week of Year,Day of Year
count,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000
mean,675.168796,675.168796,675.168796,675.168796,675.168796,675.168796,675.168796,672.086679,675.168796,675.168796,675.168796,675.168796,675.143248,675.167883,675.168796,667.025547,667.025547,675.168796,675.168796,667.025547,667.025547,667.025547,675.168796,675.168796,675.168796,675.168796,675.168796,675.168796,675.168796
std,119.081806,119.081806,119.081806,119.081806,119.081806,119.081806,119.081806,118.829300,119.081806,119.081806,119.081806,119.081806,119.073787,119.082777,119.081806,116.777333,116.777333,119.081806,119.081806,116.777333,116.777333,116.777333,119.081806,119.081806,119.081806,119.081806,119.081806,119.081806,119.081806
min,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,338.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,336.000000,336.000000,340.000000,340.000000,336.000000,336.000000,336.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000
25%,601.750000,601.750000,601.750000,601.750000,601.750000,601.750000,601.750000,598.000000,601.750000,601.750000,601.750000,601.750000,601.750000,601.750000,601.750000,595.000000,595.000000,601.750000,601.750000,595.000000,595.000000,595.000000,601.750000,601.750000,601.750000,601.750000,601.750000,601.750000,601.750000
50%,676.000000,676.000000,676.000000,676.000000,676.000000,676.000000,676.000000,674.500000,676.000000,676.000000,676.000000,676.000000,676.000000,676.000000,676.000000,670.000000,670.000000,676.000000,676.000000,670.000000,670.000000,670.000000,676.000000,676.000000,676.000000,676.000000,676.000000,676.000000,676.000000
75%,754.000000,754.000000,754.000000,754.000000,754.000000,754.000000,754.000000,750.000000,754.000000,754.000000,754.000000,754.000000,754.000000,754.000000,754.000000,747.250000,747.250000,754.000000,754.000000,747.250000,747.250000,747.250000,754.000000,754.000000,754.000000,754.000000,754.000000,754.000000,754.000000
max,1898.000000,1898.000000,1898.000000,1898.000000,1898.000000,1898.000000,1898.000000,1897.000000,1898.000000,1898.000000,1898.000000,1898.000000,1898.000000,1898.000000,1898.000000,1894.000000,1894.000000,1898.000000,1898.000000,1894.000000,1894.000000,1894.000000,1898.000000,1898.000000,1898.000000,1898.000000,1898.000000,1898.000000,1898.000000


## Histogram

In [240]:
crimes_by_date=crimes[['Date','Primary Type']].groupby(['Date']).count()


In [245]:
crimes_by_date[crimes_by_date.index=='2020-05-31']

,Primary Type
Date,
2020-05-31,1898


In [246]:
# Remove day with extreme number of crimes (looting, property damage, etc...)
crimes_by_date.drop(['2020-05-31'],inplace=True,axis=0)

In [247]:
df = crimes_by_date
df.rename(columns={'Primary Type':'Crimes per Day'}, inplace=True)
fig = px.histogram(df, x="Crimes per Day" ,nbins=40)
fig.show()

In [248]:
fig = go.Figure(data=[go.Histogram(x=df['Crimes per Day'])])
fig.show()

In [249]:
crimes_by_date
fig = go.Figure(data=go.Scatter(x=line_data.index, y=crimes_by_date['Crimes per Day'], mode='lines', marker=dict(color='green')))
fig.update_layout(title='Crimes per Day', xaxis_title='Date', yaxis_title='Crimes per Day')
fig.show()

In [182]:
y=crimes[(crimes['Date']=='2020-05-31')]
y2=y[['Date','Description']]
x=y2.groupby('Description').count()
x.rename(columns={'Date':'Count'}, inplace=True)
x.sort_values(by=['Count'])

,Count
Description,
POSSESS - CRACK,1
ATTEMPT ARSON,1
ATTEMPT ARMED - HANDGUN,1
ATTEMPT - AUTOMOBILE,1
NON-CONSENSUAL DISSEMINATION OF PRIVATE SEXUAL IMAGES,1
...,...
TO VEHICLE,67
AUTOMOBILE,76
TO PROPERTY,247


In [194]:
y=crimes[(crimes['Date']=='2020-05-31')]
y2=y[['Date','Primary Type']]
x=y2.groupby('Primary Type').count()
x.rename(columns={'Date':'Count'}, inplace=True)
x.sort_values(by=['Count'])

,Count
Primary Type,
CONCEALED CARRY LICENSE VIOLATION,1
SEX OFFENSE,2
INTERFERENCE WITH PUBLIC OFFICER,2
CRIMINAL SEXUAL ASSAULT,4
NARCOTICS,5
OFFENSE INVOLVING CHILDREN,6
HOMICIDE,19
OTHER OFFENSE,19
CRIMINAL TRESPASS,19


## Create historgrams of crimes per day

In [98]:
x=crimes[crimes['Description']=='LOOTING']
x[['Date','Description']].groupby('Date').count()

,Description
Date,
2020-03-14,1
2020-03-31,2
2020-04-30,1
2020-05-15,1
2020-05-22,1
2020-05-27,1
2020-05-29,2
2020-05-30,31
2020-05-31,289


# Create surface graph for crimes

In [168]:
crimes.shape

(739985, 30)

In [180]:
crimes['Primary Type'].unique().shape

(34,)

In [266]:
crimes[(crimes['Description']=='LOOTING') & (crimes['Year']==2020)]

,Unnamed: 0,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,Date_str,Datetime,Day of Week,Hour,Month,Week of Year,Day of Year
6917,493481,12131048,JD326907,2020-08-10,069XX S WENTWORTH AVE,3400,PUBLIC PEACE VIOLATION,LOOTING,GROCERY FOOD STORE,False,False,731,7.0,6.0,69.0,24,1176161.0,1858865.0,2020,08/17/2020 03:50:01 PM,41.768083,-87.629843,"(41.768082823, -87.629842776)",08/10/2020 01:00:00 AM,2020-08-10 01:00:00,0,1,8,33,223
7188,510285,12131390,JD326967,2020-08-10,032XX W ROOSEVELT RD,3400,PUBLIC PEACE VIOLATION,LOOTING,DEPARTMENT STORE,False,False,1134,11.0,24.0,29.0,24,1154800.0,1894524.0,2020,08/17/2020 03:50:01 PM,41.866389,-87.707189,"(41.866388583, -87.707188831)",08/10/2020 05:04:00 AM,2020-08-10 05:04:00,0,5,8,33,223
7593,537877,12131399,JD326995,2020-08-10,032XX W ROOSEVELT RD,3400,PUBLIC PEACE VIOLATION,LOOTING,DEPARTMENT STORE,False,False,1134,11.0,24.0,29.0,24,1154732.0,1894523.0,2020,08/17/2020 03:50:01 PM,41.866387,-87.707438,"(41.866387199, -87.707438494)",08/10/2020 05:00:00 AM,2020-08-10 05:00:00,0,5,8,33,223
478809,6829928,12073595,JD259121,2020-06-02,004XX S PULASKI RD,3400,PUBLIC PEACE VIOLATION,LOOTING,GAS STATION,False,False,1132,11.0,28.0,26.0,24,1149797.0,1897617.0,2020,08/31/2020 03:52:53 PM,41.874975,-87.725475,"(41.874974798, -87.725475051)",06/02/2020 06:00:00 PM,2020-06-02 18:00:00,1,18,6,23,154
520731,6884673,12153099,JD351590,2020-05-31,052XX S LAKE PARK AVE,3400,PUBLIC PEACE VIOLATION,LOOTING,SMALL RETAIL STORE,False,False,234,2.0,4.0,41.0,24,1187635.0,1870759.0,2020,10/03/2020 03:47:56 PM,41.800456,-87.587408,"(41.800455673, -87.587407515)",05/31/2020 07:01:00 PM,2020-05-31 19:01:00,6,19,5,22,152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690496,7066906,12124666,JD319505,2020-05-31,024XX W 63RD ST,3400,PUBLIC PEACE VIOLATION,LOOTING,SMALL RETAIL STORE,False,False,825,8.0,16.0,66.0,24,1161372.0,1862818.0,2020,08/06/2020 03:44:38 PM,41.779250,-87.683942,"(41.779249554, -87.683942092)",05/31/2020 01:00:00 PM,2020-05-31 13:00:00,6,13,5,22,152
693386,7070443,12127687,JD322803,2020-05-31,015XX W 69TH ST,3400,PUBLIC PEACE VIOLATION,LOOTING,SMALL RETAIL STORE,False,False,725,7.0,16.0,67.0,24,1167046.0,1858966.0,2020,08/09/2020 03:47:44 PM,41.768560,-87.663251,"(41.768559705, -87.663250604)",05/31/2020 01:00:00 AM,2020-05-31 01:00:00,6,1,5,22,152
718043,7116356,12219680,JD429598,2020-06-01,004XX E 103RD ST,3400,PUBLIC PEACE VIOLATION,LOOTING,SMALL RETAIL STORE,False,False,511,5.0,9.0,49.0,24,1180815.0,1836761.0,2020,12/17/2020 03:44:58 PM,41.707321,-87.613462,"(41.707321141, -87.613461543)",06/01/2020 02:00:00 AM,2020-06-01 02:00:00,0,2,6,23,153
725674,7124161,12233419,JD445611,2020-05-31,070XX S WESTERN AVE,3400,PUBLIC PEACE VIOLATION,LOOTING,OTHER (SPECIFY),False,False,832,8.0,17.0,66.0,24,1161594.0,1857839.0,2020,12/17/2020 03:44:58 PM,41.765582,-87.683266,"(41.765581875, -87.683266184)",05/31/2020 12:01:00 AM,2020-05-31 00:01:00,6,0,5,22,152


In [262]:
crimes.dtypes

Unnamed: 0                int64
ID                        int64
Case Number              object
Date                     object
Block                    object
IUCR                     object
Primary Type             object
Description              object
Location Description     object
Arrest                     bool
Domestic                   bool
Beat                      int64
District                float64
Ward                    float64
Community Area          float64
FBI Code                 object
X Coordinate            float64
Y Coordinate            float64
Year                      int64
Updated On               object
Latitude                float64
Longitude               float64
Location                 object
Date_str                 object
Datetime                 object
Day of Week               int64
Hour                      int64
Month                     int64
Week of Year              int64
Day of Year               int64
dtype: object